# Classify Documents

This notebook takes you through the process of training and testing a binary classification model. It uses sklearn's machine learning library. WhatEvery1Says (WE1S) has created training data for 4 kinds of news documents: obituaries, announcements, articles about the humanities, and articles about science. The training data is located in the appropriately named folders in this module. See this module's `README.md` for more information on this data and the process we used to classify documents.

## How to Use this Notebook

This notebook contains 3 sections. The first allows you to use WE1S training data to develop and evaluate a classification model on data for which classifications are already known. The second allows you use our training data to produce a model and apply it to data for which classifications are not known (i.e., the data in each of the collections outlined below). The third section allows you to save the results of your experiments to disk. We used section 1 to assist in deciding which classification models to use on our various data sets, and section 2 to train and test these models against unseen data. 

Because classification is probabilistic, the results you get when replicating our processes, even using the same data, will not exactly mirror ours.

**If you wish to replicate our experiments, you can skip directly to section 2.**

## Settings

This code block imports required Python modules and defines some variables. You will define other variables you need as you go.

In [ ]:
# Python imports
import os
from pathlib import Path
from IPython.display import display, HTML
import csv
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection, naive_bayes, svm, metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from pathlib import Path
import re

# Define paths
# before release, can delete current_dir, project_dir
# directory this notebook is in
current_dir       = %pwd
# directory of this repo on your machine
module_dir        = str(Path(current_dir).parent)

## 1. Train and Evaluate the Model on Data with Known Classifications

In this section, you load the training data included in this module and use it to develop the model you will then apply to the rest of the data in the collection you are testing. This section is provided if you wish to follow our processes exactly. Alternatively, you can just use the training data we have provided and their corresponding recommended models and skip to section 2.

This notebook utilizes k-fold cross-validation to iteratively vectorize, train, and test your model k number of times. Since classification models are probabilistic, results will differ from run to run. Using this method, you can find the average values for your evaluation metrics over k number of modeling runs. This helps you achieve a better understanding of your model's overall performance before testing it on unseen data.

Code taken from https://www.ritchieng.com/machine-learning-cross-validation/ and https://stackoverflow.com/questions/22672436/sklearn-vectorizing-in-cross-validation-for-text-classification and https://scikit-learn.org/stable/modules/cross_validation.html

### 1.A. Load Training Data

In this subsection, you load the training data csv files into a pandas dataframe, set the labels corresponding to each class, create a combined dataset with each class, and split, vectorize and model your data k number of times.

**Note on positive vs. negative observations**: The terms "positive" and "negative" are general ways to describe the two classes of documents you are working with. The positive class corresponds to the thing you are trying to identify, i.e., articles about the humanities. The negative class corresponds to the thing you are comparing the positive class to, i.e., articles not about the humanities.

Set the `classification_selection` variable below to the type of documents you want to classify and run the cell. Doing this will set the appropriate `positive_file` and `negative_file` filenames.

In [ ]:
# options are `humanities`, `science`, `announcements`, `obits`
classification_selection = 'humanities'

if classification_selection == 'humanities':
    positive_file = 'humanities/hum-positive.csv'
    negative_file = 'humanities/hum-negative.csv'
elif classification_selection == 'science':
    positive_file = 'science/sci-positive.csv'
    negative_file = 'science/sci-negative.csv'
elif classification_selection == 'announcements':
    positive_file = 'announcements/announce-positive.csv'
    negative_file = 'announcements/announce-negative.csv'
elif classification_selection == 'obits':
    positive_file = 'obits/obits-positive.csv'
    negative_file = 'obits/obits-negative.csv'

Then import this data to a dataframe.

In [ ]:
# import training data to pandas dataframes
positive = pd.read_csv(positive_file)
negative = pd.read_csv(negative_file)


**Note on labeling**: Since there are two labels for our data -- positive or negative -- this notebook represents those labels as numbers (0 = negative and 1 = positive) from the start. In cases where you are dealing with data that you want to store with its labels in text form, or where there are multiple labels, you may want instead to represent these labels as text values, i.e., 'obit' and 'not obit'. However, the model needs these labels to be in numerical form, so if you store them in text form, you would need to convert them to numbers before modeling. You can use sklearn's `LabelEncoder` class to automatically do this conversion before vectorization, i.e.,:

`Corpus['label_num'] = Corpus.label.map({'control':0, 'obit':1})`

But again, this notebook skips this step and just stores them as numbers from the beginning.

In [ ]:
# add labels, positive = 1 and negative = 0
positive['label'] = 1
negative['label'] = 0

# Create a combined dataframe, with obits and control
Corpus = positive.append(negative, ignore_index=True)

# Check to make sure everything above worked
Corpus

**Split data into X and Y sets**: Here you split your data into X and Y sets before splitting into training and testing sets (which is done automatically during the cross-validation process). The X and Y sets represent a splitting of 2 columns off from the `Corpus` dataframe created in the code block above. Note the composition of the `Corpus` dataframe above. It contains 4 columns: index (unnamed), filename, text, label. When you create your `X` and `Y` sets, you put the text column into `X`, and the label column into `Y`. This separates the data you want your classifier to learn, the text column, from its class label, the label column.

In [ ]:
# split into X and Y sets
X = Corpus['text']
Y = Corpus['label']

### 1.B. Develop the Model

Now you will use the known data you loaded above to develop your classification model. After a series of experiments, we decided to use 2 different classification models. We used a logistic regression model (raw count vectorization) to classify articles about the humanities and articles about science, and we used an SVM model (tfidf vectorization) to classify obituaries and announcements. Code implementing both kinds of models is included below.

**Logistic Regression**: First, you create what sklearn calls a Pipeline, which combines vectorization with modeling. This way, each time a modeling run is initiated, the training data for that run is fit and transformed into vectors, and then the testing set is transformed into vectors (but not fit). This ensures that your testing data is never used to fit a vectorization model.

Remember, you are still using data for which all classification labels are known (i.e., the training data). The cross-validation process splits this data up into 10 different sets of ad-hoc "training" and "testing" data in order to get a sense of the model's average performance over a series of runs.

In [ ]:
# Create a logistic regression pipeline
clf_logreg = Pipeline([('vect', CountVectorizer()), ('linear_model', LogisticRegression())])
    

In [ ]:
# Do the modeling, cv = 10 means we want to train/test split and model 10 times. 
# This is an example of 10-fold cross-validation.

%time scores_logreg = cross_val_score(clf_logreg, X, Y, cv=10)
scores_logreg

**Evaluation:** Different metrics for evaluating the model are included below. All scores are averages over 10 runs (or whatever `cv` was set to above).

In [ ]:
# Average accuracy across the 10 runs
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_logreg.mean(), scores_logreg.std() * 2))

In [ ]:
scores_logreg = cross_val_score(clf_logreg, X, Y, cv=10, scoring='roc_auc')
scores_logreg

print("AUC: %0.2f (+/- %0.2f)" % (scores_logreg.mean(), scores_logreg.std() * 2))

In [ ]:
scores_logreg = cross_val_score(clf_logreg, X, Y, cv=10, scoring='recall')
scores_logreg

print("Sensitivity/Recall score: %0.2f (+/- %0.2f)" % (scores_logreg.mean(), scores_logreg.std()))
                                                       

In [ ]:
scores_logreg = cross_val_score(clf_logreg, X, Y, cv=10, scoring='f1')
scores_logreg

print("F1 score: %0.2f (+/- %0.2f)" % (scores_logreg.mean(), scores_logreg.std() * 2))

**SVM:** First, we create what sklearn calls a Pipeline, which combines vectorization with modeling. This way, each time a modeling run is initiated, the training data for that run is vectorized. This ensures, like above, that our testing data is never used to fit a vectorization model.

Remember, you are still using data for which all classification labels are known (i.e., the training data). The cross-validation process splits this data up into 10 different sets of ad-hoc "training" and "testing" data in order to get a sense of the model's average performance over a series of runs.

In [ ]:
# Create a SVM pipeline
clf_SVM = Pipeline([('vect', TfidfVectorizer()), ('svm', svm.SVC(C=1.0, kernel='linear'))])

In [ ]:
# Do the modeling, cv = 10 means we want to train/test split and model 10 times. 
# This is an example of 10-fold cross-validation.

%time scores_SVM = cross_val_score(clf_SVM, X, Y, cv=10)
scores_SVM

**Evaluation:** Different metrics for evaluating the model are included below. All scores are averages over 10 runs (or whatever `cv` was set to above).

In [ ]:
# Average accuracy across the 10 runs
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_SVM.mean(), scores_SVM.std() * 2))

In [ ]:
scores_SVM = cross_val_score(clf_SVM, X, Y, cv=10, scoring='roc_auc')
scores_SVM

print("AUC: %0.2f (+/- %0.2f)" % (scores_SVM.mean(), scores_SVM.std() * 2))

In [ ]:
scores_SVM = cross_val_score(clf_SVM, X, Y, cv=10, scoring='recall')
scores_SVM

print("Sensitivity/Recall score: %0.2f (+/- %0.2f)" % (scores_SVM.mean(), scores_SVM.std() * 2))

In [ ]:
scores_SVM = cross_val_score(clf_SVM, X, Y, cv=10, scoring='precision')
scores_SVM

print("Precision score: %0.2f (+/- %0.2f)" % (scores_SVM.mean(), scores_SVM.std() * 2))

In [ ]:
scores_SVM = cross_val_score(clf_SVM, X, Y, cv=10, scoring='f1')
scores_SVM

print("F1 score: %0.2f (+/- %0.2f)" % (scores_SVM.mean(), scores_SVM.std() * 2))

## 2. Test the Model on Unseen Data

After we have developed a model we are happy with, we can test it on data for which we do not know the classification. 

The first step is to train the model we have developed and tuned on *all* of our training data. Above, we split our training data into ad-hoc training and testing sets for the purposes of developing our model. Now, however, we will train our model on all of our data for which the classification labels are known. We won't split this data into training and testing sets because it all becomes our training set, and the rest of the data in the collection we are testing becomes our testing set.

### 2.A. Load Training Data

First, as above, you load the training data csv files into a pandas dataframe, set the labels corresponding to each class, create a combined dataset with each class, and split, vectorize and model your data.

**Note on positive vs. negative observations**: The terms "positive" and "negative" are general ways to describe the two classes of documents you are working with. The positive class corresponds to the thing you are trying to identify, i.e., articles about the humanities. The negative class corresponds to the thing you are comparing the positive class to, i.e., articles not about the humanities.

Set the `classification_selection` variable below to the type of documents you want to classify and run the cell. Doing this will set the appropriate `positive_file` and `negative_file` filenames.

In [ ]:
# options are `humanities`, `science`, `announcements`, `obits`
classification_selection = 'humanities'

if classification_selection == 'humanities':
    positive_file = 'humanities/hum-positive.csv'
    negative_file = 'humanities/hum-negative.csv'
elif classification_selection == 'science':
    positive_file = 'science/sci-positive.csv'
    negative_file = 'science/sci-negative.csv'
elif classification_selection == 'announcements':
    positive_file = 'announcements/announce-positive.csv'
    negative_file = 'announcements/announce-negative.csv'
elif classification_selection == 'obits':
    positive_file = 'obits/obits-positive.csv'
    negative_file = 'obits/obits-negative.csv'

Then import this data to a dataframe.

In [ ]:
# import training data to pandas dataframes
positive = pd.read_csv(positive_file)
negative = pd.read_csv(negative_file)

**Note on labeling**: Since there are two labels for our data -- positive or negative -- this notebook represents those labels as numbers (0 = negative and 1 = positive) from the start. In cases where you are dealing with data that you want to store with its labels in text form, or where there are multiple labels, you may want instead to represent these labels as text values, i.e., 'obit' and 'not obit'. However, the model needs these labels to be in numerical form, so if you store them in text form, you would need to convert them to numbers before modeling. You can use sklearn's `LabelEncoder` class to automatically do this conversion before vectorization, i.e.,:

`Corpus['label_num'] = Corpus.label.map({'control':0, 'obit':1})`

But again, this notebook skips this step and just stores them as numbers from the beginning.

In [ ]:
# add labels, positive = 1 and negative = 0
positive['label'] = 1
negative['label'] = 0

# Create a combined dataframe, with obits and control
Corpus = positive.append(negative, ignore_index=True)

# Check to make sure everything above worked
Corpus

**Split data into X and Y sets**: Here, as in section 1, you split your data into X and Y sets. The X and Y sets represent a splitting of 2 columns off from the `Corpus` dataframe created in the code block above. Note the composition of the `Corpus` dataframe above. It contains 4 columns: index (unnamed), filename, text, label. When you create your `X` and `Y` sets, you put the text column into `X`, and the label column into `Y`. This separates the data you want your classifier to learn, the text column, from its class label, the label column.

In [ ]:
# split into X and Y sets
X = Corpus['text']
Y = Corpus['label']

### 2.B. Train the Model

Now you will use the known data you loaded above to produce your classification model. After a series of experiments, we decided to use 2 different classification models. We used a logistic regression model (raw count vectorization) to classify articles about the humanities and articles about science, and we used an SVM model (tfidf vectorization) to classify obituaries and announcements. Code implementing both kinds of models is included below.

**Logistic Regression Model:** For use to classify articles about the humanities and articles about science. 

In [ ]:
# Initialize a vectorizer
# use `CountVectorizer()` for the logistic regression model
# and `TfidfVectorizer()` for the SVM model
vect = CountVectorizer()

In [ ]:
# apply vectors to X -- no need to split into training and testing because all of human classified data is now
# training set; testing set will be the rest of the data in the collection
X_dtm = vect.fit_transform(X)


In [ ]:
# fit training dataset on logreg classifier
logreg = LogisticRegression()

%time logreg.fit(X_dtm, Y)

**SVM:** For use classifying announcements and obituaries.

In [ ]:
# Initialize a vectorizer
# use `CountVectorizer()` for the logistic regression model
# and `TfidfVectorizer()` for the SVM model
vect = TfidfVectorizer()

In [ ]:
# apply vectors to X -- no need to split into training and testing because all of human classified data is now
# training set; testing set will be the rest of the data in the collection
X_dtm = vect.fit_transform(X)


In [ ]:
# fit training dataset on SVM classifier
SVM = svm.SVC(C=1.0, kernel='linear')

%time SVM.fit(X_dtm, Y)

### 2.C. Load the Unseen Data

Now you will select data from the collection that the model hasn't already seen. To do that, you need to check if any of the documents included in the training data exist in the collection you are testing. If so, you won't include them in your test. The code below creates a list of the filenames of the documents included in your training set. Then it checks this list against the collection's `doc-terms` file. It writes the filenames of the documents in the collection you are testing that aren't in the training data to a separate text file, and it writes the filenames of those documents that are included in the training data to separate positive and negative text files.

First, however, you need to define the filepaths for the files you want to load and create. Defining the `collection_selection` variable will automatically set the others described below:

* `collection_selection` = The collection you are testing. The 4 collections we ran classification experiments on for this article include `c14`, `c18`, `c20`, `c21`.
* `collection` = the doc-terms file corresponding to the collection you have selected.
* `unseen_filenames` = the text file that will list the filenames of the documents from the above collection that are not included in the training data. This file will be stored in the appropriate classification type folder (`humanities`, `science`, `announcements`, `obits`).
* `classified_p_filenames` = the text file that will list the filenames from the collection that are in the positive training data. This file will be stored in the appropriate classification type folder (`humanities`, `science`, `announcements`, `obits`).
* `classified_n_filenames` = the text file that will list the filenames from the collection that are in the negative training data. This file will be stored in the appropriate classification type folder (`humanities`, `science`, `announcements`, `obits`).
* `csv_input` = the csv file you will use for input in your text classification experiment. It will contain all of the documents in the collection you are testing that aren't already included in the training data. This file will be stored in the appropriate classification type folder (`humanities`, `science`, `announcements`, `obits`).

In [ ]:
# options are 'c14', 'c18', 'c20', 'c21'
collection_selection = 'c14'

collection = module_dir + '/data/doc-terms/' + collection_selection + '-doc-terms.txt'
unseen_filenames = classification_selection + '/' + collection_selection + '-hum-remaining-files.txt'
classified_p_filenames = classification_selection + '/' + collection_selection + '-hum-positive-classified.txt'
classified_n_filenames = classification_selection + '/' + collection_selection + '-hum-negative-classified.txt'
csv_input = classification_selection + '/' + collection_selection + '-unseen.csv'

Create new lists described above.

In [ ]:
# create lists of filenames from positive and negative training data
filenames_positive = []
filenames_negative = []

with open(positive_filenames) as f:
    for row in f:
        row = row.strip()
        filenames_positive.append(row)

with open(negative_filenames) as f:
    for row in f:
        row = row.strip()
        filenames_negative.append(row)

# write the filenames of documents in the collection that are NOT in training data to disk 
# ('unseen_fiilenames')
# and write the filenames of documents in the collection that ARE in training data to disk 
# ('classified_p_filenames' and 'classified_n_filenames')

with open(unseen_filenames, 'w') as f_unseen, open(classified_p_filenames, 'w') as f_cp, open(classified_n_filenames, 'w') as f_cn:
    with open(collection) as f_collection:
        for file in f_collection:
            file = file.strip()
            file = file.split(' ')
            file = file[0]
            if file not in filenames_positive and file not in filenames_negative:
                f_unseen.write(file + '\n')
            elif file in filenames_positive:
                f_cp.write(file + '\n')
            elif file in filenames_negative:
                f_cn.write(file + '\n')


Now you are going to use the list of filenames from the collection that are not included in the training data to create the csv input file you need for the classification test.

In [ ]:
# Define the function you need

def get_bags(collection, csv_input, unseen_filenames):
    '''Convert collection doc-terms.txt file to csv file needed for classification, including 
    only specified files from the collection.'''
    filenames_list = []
    with open(unseen_filenames) as f1:
        for item in f1:
            item = item.strip()
            filenames_list.append(item)
    with open(csv_input, 'w') as cf:
        csv_writer = csv.writer(cf, delimiter = ',')
        csv_writer.writerow(['filename', 'text'])
        with open(collection) as f:
            for row in f:
                row = row.strip()
                row = row.split(' ')
                filename = row[0]
                if filename in filenames_list:
                    x = len(row)
                    bow = ' '.join(row[2:x])
                    csv_writer.writerow([filename, bow])

In [ ]:
# implement the above function: use get_bags to create csv input for classificatione experiment

get_bags(collection, csv_input, unseen_filenames)


Next, you will use the csv file you just created to load the unseen data into a dataframe for analysis. If you did not run the `get_bags` function above to create a new collection csv file, you need to set the `csv_input` variable to the csv file you would like to load (i.e., the csv file for the collection you are testing). To do that, uncomment the `csv_input` line below.

In [ ]:
# import data and create dataframe
# csv_input = 
unseen = pd.read_csv(csv_input)


In [ ]:
# check dataframe
unseen

**Split data into X set but not Y**: Here you split your data into an X set, but not a Y set as above (because you don't know the labels). The `Xnew` dataframe contains the text from each document you will test in your collection. The classifier will then predict the label of each document.

In [ ]:
# split into Xnew set
Xnew = unseen['text']

# check for empty rows
Xnew = Xnew.dropna()

### 2.D. Test the Model

As above, code for testing your data using a logistice regression model and/or an SVM model is included below.

**Logistic Regression Model:** In the cell below, you will vectorize the data, but you don't fit the model because it's already fit on training data (under section 2, "Train the Data" above). You also do not need to initialize a new vectorizer because you want to use the same one you used above. Then you will test your model.

In [ ]:
# vectorize the test data
Xnew_dtm = vect.transform(Xnew)

In [ ]:
# predict classes of unseen data using logreg
%time predictions_logreg_new = logreg.predict(Xnew_dtm)

In the cells below, you can preview your results and see the total number of documents predicted for each class.

In [ ]:
# see document test of those documents predicted to be positive
positive = Xnew[predictions_logreg_new ==1]

print(positive)

In [ ]:
# see document test of those documents predicted to be negative
negative = Xnew[predictions_logreg_new ==0]

print(negative)

**SVM Model:** In the cell below, you will vectorize the data, but you don't fit the model because it's already fit on training data (under section 2, "Train the Data" above). You also do not need to initialize a new vectorizer because you want to use the same one you used above. Then you will test your model.

In [ ]:
# vectorize the test data
Xnew_dtm = vect.transform(Xnew)

In [ ]:
# predict classes of unseen data using svm
%time predictions_svm_new = SVM.predict(Xnew_dtm)

In the cells below, you can preview your results and see the total number of documents predicted for each class.

In [ ]:
# see document test of those documents predicted to be positive
positive = Xnew[predictions_svm_new ==1]

print(positive)

In [ ]:
# see document test of those documents predicted to be negative
negative = Xnew[predictions_svm_new ==0]

print(negative)

## 3. Save Your Results to Disk

You can save the results of your experiments to text files using the code below. First, you need to define the following variables:

* `positive_file` = the file you want to write the filenames (and if you wish, search keywords) predicted by the model to be positive results. This file will be stored in the `results` folder in this module.
* `negative_file` = the file you want to write the filenames (and if you wish, search keywords) predicted by the model to be negative results. This file will be stored in the `results` folder in this module.
* `keyword` = this is set to `False` by default. If you change its value to `True`, the function will print the search keyword (e.g., "humanities, "science," or "comparison") used in collecting each document to your results file.

The code below uses the results `positive` and `negative` results arrays created above to write results to disk.

In [ ]:
# Define variables and function

positive_file = 'results/' + collection_selection + classification_selection + '-positive-results.txt'
negative_file = 'results/' + collection_selection + classification_selection + '-negative-results.txt'
keyword = False

def save_results(positive, negative, positive_file, negative_file, keyword):
    with open(positive_file, 'w') as pf:
        for index, text in positive.iteritems():
            filename = unseen.loc[index, 'filename']
            if keyword == True:
                term = ''
                if '_bodypluralhumanitiesorhleadpluralhumanities_' in filename:
                    term = 'humanities'
                elif '_bodyliberalpre1pluralartsorhleadliberalpre1pluralarts_' in filename:
                    term = 'liberal arts'
                elif '_bodysciencesorhleadsciences_' in filename:
                    term = 'science'
                elif '_sayandnothumanities_' in filename or '_goodandnothumanities_' in filename or '_personandnothumanities_' in filename:
                    term = 'comparison' 
                else:
                    term = 'unknown'
                pf.write(filename + ',' + term + '\n')
            else:
                pf.write(filename + '\n')
    with open(negative_file, 'w') as nf:
        for index, text in negative.iteritems():
            filename = unseen.loc[index, 'filename']
            if keyword == True:
                term = ''
                if '_bodypluralhumanitiesorhleadpluralhumanities_' in filename:
                    term = 'humanities'
                elif '_bodyliberalpre1pluralartsorhleadliberalpre1pluralarts_' in filename:
                    term = 'liberal arts'
                elif '_bodysciencesorhleadsciences_' in filename:
                    term = 'science'
                elif '_sayandnothumanities_' in filename or '_goodandnothumanities_' in filename or '_personandnothumanities_' in filename:
                    term = 'comparison' 
                else:
                    term = 'unknown'
                nf.write(filename + ',' + term + '\n')
            else:
                nf.write(filename + '\n')

In [ ]:
# do the thing
save_results(positive, negative, positive_file, negative_file, keyword)